# Lead Calculation with Rule-Based Classification

## Business Problem



A game company wants to create new level-based customer definitions (personas) by using some features of its customers, and to create segments according to these new customerdefinitions and to estimate how much the new customers can earn on average according to these segments.

### For example:
It is desired to determine how much a 25-year-old male user from Turkey who is an IOS user can earn on average.

## Dataset Story

The Persona.csv dataset contains the prices of the products sold by an international game company and some demographic information of the users who buy these products. The data set consists of records created in each sales transaction. This means table not singularized. In other words, a user with certain demographic characteristics may have made more than one purchase.

Price: Customer's spending amount

Source: The type of device the customer is connecting to

Sex: Gender of the client

Country: Country of the customer

Age: Customer's age


In [1]:
#Necessary libraries are imported.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../input/persona/persona.csv")


# 1.DATA OVERVİEW

In [3]:
#The number of unique values for each variable in the dataset is shown.
df.shape

(5000, 5)

In [4]:
#The first 5 rows of the dataset were looked at.
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [5]:
#The last 5 rows of the dataset were viewed.
df.tail()

,PRICE,SOURCE,SEX,COUNTRY,AGE
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [6]:
#Variable types and numbers of the data set were examined.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [7]:
#It was checked for missing data in the dataset.
df.isnull().sum()

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [8]:
#The statistical properties of the columns of the data set containing numeric data were checked.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


In [9]:
#The number of unique values for each variable in the dataset was looked at.
df.nunique()

PRICE       6
SOURCE      2
SEX         2
COUNTRY     6
AGE        46
dtype: int64

# 2.DATA MANIPULATION

In [10]:
"""
The dataset variables are examined to ensure their data types. Some numerical variables can be treated as categorical if they have
low unique value counts and similarly some categorical variables can have high unique value counts so they are treated as cardinal
variables. For this application variable_catcher function is written
"""

def catcher(dataframe, cat_threshold = 5, car_threshold = 10):
    
    # To catch categorical variables
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtype) in ["object","category","bool"]]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_threshold and 
                   str(dataframe[col].dtype) in ["int64","float64"]]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_threshold and 
                   str(dataframe[col].dtype) in ["object","category"]]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    
    # To catch numerical variables
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["int64", "float64"]]
    num_cols = [col for col in num_cols if col not in cat_cols]
    
    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = catcher(df)

Observations: 5000
Variables: 5
cat_cols: 3
num_cols: 2
cat_but_car: 0
num_but_cat: 0


In [11]:
#It shows the frequency values of the type of device that the customer is connected to.
df["SOURCE"].value_counts(normalize = True)

android    0.5948
ios        0.4052
Name: SOURCE, dtype: float64

In [12]:
#The number of sales by country is shown.
df.groupby("COUNTRY").agg ({"PRICE":"count"})

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


In [13]:
#The average amount and total earnings of the countries from the sale are shown.
df.groupby("COUNTRY",).agg ({"PRICE":["sum","mean"]})

PRICE           
           sum       mean
COUNTRY                  
bra      51354  34.327540
can       7730  33.608696
deu      15485  34.032967
fra      10177  33.587459
tur      15689  34.787140
usa      70225  34.007264

In [14]:
## An aggregated dataframe is created that shows mean price values grouped by ("COUNTRY","SOURCE","SEX","AGE") variables.
agg_df = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg ({"PRICE":"mean"}).copy()
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

In [15]:
#Convert the names in the index to variable names.
agg_df.reset_index(inplace=True)
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,female,15,38.714286
1,bra,android,female,16,35.944444
2,bra,android,female,17,35.666667
3,bra,android,female,18,32.255814
4,bra,android,female,19,35.206897
...,...,...,...,...,...
343,usa,ios,male,42,30.250000
344,usa,ios,male,50,39.000000
345,usa,ios,male,53,34.000000
346,usa,ios,male,55,29.000000


In [16]:
#Converted age variable to categorical variable and added to agg_df.
agg_df["AGE CAT"] = pd.cut(agg_df.AGE, bins = [0,18,23,30,40,70], labels = ["0_18","19_23","24_30","31_40","41_70"])
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE CAT
0,bra,android,female,15,38.714286,0_18
1,bra,android,female,16,35.944444,0_18
2,bra,android,female,17,35.666667,0_18
3,bra,android,female,18,32.255814,0_18
4,bra,android,female,19,35.206897,19_23
...,...,...,...,...,...,...
343,usa,ios,male,42,30.250000,41_70
344,usa,ios,male,50,39.000000,41_70
345,usa,ios,male,53,34.000000,41_70
346,usa,ios,male,55,29.000000,41_70


In [17]:
#
categorical_types =[col for col in agg_df.columns if agg_df[col].dtype.name in ["category","object"]]
agg_df["customer_level_based"] = agg_df[categorical_types].apply("_".join,axis=1).str.upper()
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE CAT,customer_level_based
0,bra,android,female,15,38.714286,0_18,BRA_ANDROID_FEMALE_0_18
1,bra,android,female,16,35.944444,0_18,BRA_ANDROID_FEMALE_0_18
2,bra,android,female,17,35.666667,0_18,BRA_ANDROID_FEMALE_0_18
3,bra,android,female,18,32.255814,0_18,BRA_ANDROID_FEMALE_0_18
4,bra,android,female,19,35.206897,19_23,BRA_ANDROID_FEMALE_19_23
...,...,...,...,...,...,...,...
343,usa,ios,male,42,30.250000,41_70,USA_IOS_MALE_41_70
344,usa,ios,male,50,39.000000,41_70,USA_IOS_MALE_41_70
345,usa,ios,male,53,34.000000,41_70,USA_IOS_MALE_41_70
346,usa,ios,male,55,29.000000,41_70,USA_IOS_MALE_41_70


In [18]:
## To check the labeling consistency following code can be implemented
agg_df.groupby("AGE CAT").agg({"AGE":['min','max']})

AGE    
        min max
AGE CAT        
0_18     15  18
19_23    19  23
24_30    24  30
31_40    31  40
41_70    41  66

In [19]:
#Singularization of customers was done.
agg_df = agg_df.groupby("customer_level_based").agg({"PRICE" : "mean"}).reset_index()
agg_df

,customer_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_70,36.737179
...,...,...
104,USA_IOS_MALE_0_18,33.983495
105,USA_IOS_MALE_19_23,34.901872
106,USA_IOS_MALE_24_30,34.838143
107,USA_IOS_MALE_31_40,36.206324


In [20]:
# New customer variable is grouped into 4 segments and assigned to a new variable called "SEGMENT
agg_df["segment"] = pd.qcut(agg_df.PRICE, 4, labels = ["D","C","B","A"])
agg_df

,customer_level_based,PRICE,segment
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_70,36.737179,A
...,...,...,...
104,USA_IOS_MALE_0_18,33.983495,C
105,USA_IOS_MALE_19_23,34.901872,B
106,USA_IOS_MALE_24_30,34.838143,B
107,USA_IOS_MALE_31_40,36.206324,A


In [21]:
# The segments are shown by their mean, max, sum values 
agg_df.groupby("segment").agg({"PRICE": ["mean","sum","max"]})

PRICE                        
              mean          sum        max
segment                                   
D        29.206780   817.789833  32.333333
C        33.509674   904.761209  34.077340
B        34.999645   944.990411  36.000000
A        38.691234  1044.663328  45.428571

# 3.LEAD CALCULATION

What is the segment of a 33 year old Turkish woman who uses Android and how much profit the company gains from her on average?

In [22]:
new_user="TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customer_level_based"] == new_user]

,customer_level_based,PRICE,segment
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


What is the segment of a 35 year old French woman who uses iOS and how much profit the company gains from her on average?

In [23]:
new_user="FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customer_level_based"] == new_user]

,customer_level_based,PRICE,segment
63,FRA_IOS_FEMALE_31_40,32.818182,C
